In [ ]:
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

In [ ]:
df = pd.read_csv('F:/Python/project 650/_CDI.Training.raw.csv')

In [ ]:
df.shape

(1294, 42)

In [ ]:
def cstat(df, sortstyle='Feature', return_lists = False, show_table = True, nan_threshold = 50, zero_threshold = 50, rnd_lvl=2):
    results = []

    for column in df:
        dtype = str(df[column].dtype)
        unique_count = df[column].nunique()

        noNan_count = df[column].count()
        Nan_count = df[column].isna().sum()
        dupe_counts = df[column].value_counts()
        dupes = dupe_counts[dupe_counts > 1]
        num_dupes = dupes.count()

        percent_nan = round(df[column].isna().mean() * 100, rnd_lvl)
        percent_zero = round((df[column] == 0).mean() * 100, rnd_lvl)

        min_value = "."
        max_value = "."
        range_value = "."
        mean_value = "."
        median_value = "."
        std_dev = "."
        kurtosis = "."
        skewness = "."

        if pd.api.types.is_numeric_dtype(df[column]):
            min_value = round(df[column].min(), rnd_lvl)
            max_value = round(df[column].max(), rnd_lvl)
            mean_value = round(df[column].mean(), rnd_lvl)
            median_value = round(df[column].median(), rnd_lvl)
            std_dev = round(df[column].std(), rnd_lvl)
            kurtosis = round(df[column].kurtosis(), rnd_lvl)
            skewness = round(df[column].skew(), rnd_lvl)
            if pd.isna(kurtosis) or not isinstance(kurtosis, (int, float)): #bc if there's too many nans, kurtosis returns a string
                kurtosis = "."
            if pd.isna(skewness) or not isinstance(skewness, (int, float)): #same for skew
                skewness = "."

        results.append({
            "Feature": column,
            "Dtype": dtype,
            "# Count": noNan_count,
            "# NaN": Nan_count,
            "# Unique": unique_count,
            "# Dupes": num_dupes,
            "Min": min_value,
            "Max": max_value,
            "% NaN": percent_nan,
            "% Zero": percent_zero,
            "Mean": mean_value,
            "Median": median_value,
            "Std Dev": std_dev,
            "Kurtosis (3)": kurtosis,
            "Skew (0)": skewness
        })

    results_df = pd.DataFrame(results)

    if sortstyle == 'Feature':
        results_df = results_df.sort_values(by=sortstyle)
    elif sortstyle == 'Dtype':
        results_df = results_df.sort_values(by=[sortstyle, 'Feature'], ascending =[False, True])
    else:
        results_df = results_df.sort_values(by=[sortstyle, 'Feature'])

    pd.options.display.float_format = f'{{:.{rnd_lvl}f}}'.format

    if show_table: display(results_df)

    if return_lists:
        results_df = results_df[results_df['# Unique'] > 2]

        nancols2drop = results_df[results_df['% NaN'] > nan_threshold]['Feature'].tolist()
        nancols2drop_str = ', '.join([f"'{s}'" for s in nancols2drop])
        print(f"nancols2drop = [{nancols2drop_str}] #nan threshold: {nan_threshold}%")

        zerocols2drop = results_df[results_df['% Zero'] > zero_threshold]['Feature'].tolist()
        zerocols2drop_str = ', '.join([f"'{s}'" for s in zerocols2drop])
        print(f"zerocols2drop = [{zerocols2drop_str}] #zero threshold: {zero_threshold}%")

        skew_df = results_df[results_df['Skew (0)'] != '.']
        cols2normalize = skew_df[(skew_df['Skew (0)'] > 1) | (skew_df['Skew (0)'] < -1)]['Feature'].tolist()
        cols2normalize_str = ', '.join([f"'{s}'" for s in cols2normalize])
        print(f"cols2normalize = [{cols2normalize_str}]")

        kurt_df = results_df[results_df['Kurtosis (3)'] != '.']
        cols2winsorize = kurt_df[kurt_df['Kurtosis (3)'] > 5]['Feature'].tolist()
        cols2winsorize_str = ', '.join([f"'{s}'" for s in cols2winsorize])
        print(f"cols2winsorize = [{cols2winsorize_str}]")

        all_numeric_columns = df.select_dtypes(include=['number']).columns

        binary_cols = [col for col in all_numeric_columns if df[col].nunique() == 2]
        binarycols_str = ', '.join([f"'{s}'" for s in binary_cols])
        print(f"binary_cols = [{binarycols_str}]")

        numeric_cols = [col for col in all_numeric_columns if df[col].nunique() > 2]
        numericcols_str = ', '.join([f"'{s}'" for s in numeric_cols])
        print(f"numeric_cols = [{numericcols_str}]")

        object_cols = df.select_dtypes(include=['object']).columns
        objectcols_str = ', '.join([f"'{s}'" for s in object_cols])
        print(f"object_cols = [{objectcols_str}]")

        return nancols2drop, zerocols2drop, cols2normalize, cols2winsorize, binary_cols, numeric_cols, object_cols

In [ ]:
cstat(df,'Dtype')

,Feature,Dtype,# Count,# NaN,# Unique,# Dupes,Min,Max,% NaN,% Zero,Mean,Median,Std Dev,Kurtosis (3),Skew (0)
0,SUBJECT_ID,int64,1294,0,1294,0,21.00,99973.00,0.00,0.00,36133.12,27062.50,28643.37,-0.75,0.73
2,age,int64,1294,0,71,70,20.00,300.00,0.00,0.00,85.62,71.00,65.35,6.49,2.80
6,chronic_ischemic_heart_disease,int64,1294,0,2,2,0.00,1.00,0.00,71.72,0.28,0.00,0.45,-1.07,0.97
5,chronic_kidney_disease,int64,1294,0,2,2,0.00,1.00,0.00,71.64,0.28,0.00,0.45,-1.08,0.96
4,diabetes,int64,1294,0,2,2,0.00,1.00,0.00,65.53,0.34,0.00,0.48,-1.57,0.65
1,gender_binaried,int64,1294,0,2,2,0.00,1.00,0.00,48.69,0.51,1.00,0.50,-2.00,-0.05
3,inhospital_death,int64,1294,0,2,2,0.00,1.00,0.00,79.13,0.21,0.00,0.41,0.06,1.44
8,albumin,float64,1097,197,39,35,1.00,5.00,15.22,0.00,2.96,3.00,0.68,-0.34,0.07
7,anion_gap,float64,1294,0,35,30,4.00,41.00,0.00,0.00,16.23,16.00,4.74,3.92,1.40
9,bicarbonate,float64,1293,1,38,35,5.00,52.00,0.08,0.00,23.79,24.00,5.36,1.74,0.07


In [ ]:
df.drop('SUBJECT_ID', axis=1, inplace=True)
df.drop('age', axis=1, inplace=True)

In [ ]:
cstat(df,'Dtype')

,Feature,Dtype,# Count,# NaN,# Unique,# Dupes,Min,Max,% NaN,% Zero,Mean,Median,Std Dev,Kurtosis (3),Skew (0)
4,chronic_ischemic_heart_disease,int64,1294,0,2,2,0.00,1.00,0.00,71.72,0.28,0.00,0.45,-1.07,0.97
3,chronic_kidney_disease,int64,1294,0,2,2,0.00,1.00,0.00,71.64,0.28,0.00,0.45,-1.08,0.96
2,diabetes,int64,1294,0,2,2,0.00,1.00,0.00,65.53,0.34,0.00,0.48,-1.57,0.65
0,gender_binaried,int64,1294,0,2,2,0.00,1.00,0.00,48.69,0.51,1.00,0.50,-2.00,-0.05
1,inhospital_death,int64,1294,0,2,2,0.00,1.00,0.00,79.13,0.21,0.00,0.41,0.06,1.44
6,albumin,float64,1097,197,39,35,1.00,5.00,15.22,0.00,2.96,3.00,0.68,-0.34,0.07
5,anion_gap,float64,1294,0,35,30,4.00,41.00,0.00,0.00,16.23,16.00,4.74,3.92,1.40
7,bicarbonate,float64,1293,1,38,35,5.00,52.00,0.08,0.00,23.79,24.00,5.36,1.74,0.07
8,bilirubin_total,float64,1098,196,92,43,0.00,82.80,15.15,0.15,1.34,0.50,3.88,202.51,11.94
20,bun,float64,1294,0,130,103,2.00,240.00,0.00,0.00,34.52,26.00,26.71,5.36,1.95


In [ ]:
#replace negative values with nans bc they're impossible, e.g. negative blood pressure; patient prolly wasn't hooked in yet
negative_count = 0
for col in df.columns:
    negative_count += (df[col] < 0).sum()
    df[col] = df[col].where(df[col] >= 0, other=np.nan)
print(negative_count)

16


In [ ]:
cstat(df,'Dtype')

,Feature,Dtype,# Count,# NaN,# Unique,# Dupes,Min,Max,% NaN,% Zero,Mean,Median,Std Dev,Kurtosis (3),Skew (0)
4,chronic_ischemic_heart_disease,int64,1294,0,2,2,0.00,1.00,0.00,71.72,0.28,0.00,0.45,-1.07,0.97
3,chronic_kidney_disease,int64,1294,0,2,2,0.00,1.00,0.00,71.64,0.28,0.00,0.45,-1.08,0.96
2,diabetes,int64,1294,0,2,2,0.00,1.00,0.00,65.53,0.34,0.00,0.48,-1.57,0.65
0,gender_binaried,int64,1294,0,2,2,0.00,1.00,0.00,48.69,0.51,1.00,0.50,-2.00,-0.05
1,inhospital_death,int64,1294,0,2,2,0.00,1.00,0.00,79.13,0.21,0.00,0.41,0.06,1.44
6,albumin,float64,1097,197,39,35,1.00,5.00,15.22,0.00,2.96,3.00,0.68,-0.34,0.07
5,anion_gap,float64,1294,0,35,30,4.00,41.00,0.00,0.00,16.23,16.00,4.74,3.92,1.40
7,bicarbonate,float64,1293,1,38,35,5.00,52.00,0.08,0.00,23.79,24.00,5.36,1.74,0.07
8,bilirubin_total,float64,1098,196,92,43,0.00,82.80,15.15,0.15,1.34,0.50,3.88,202.51,11.94
20,bun,float64,1294,0,130,103,2.00,240.00,0.00,0.00,34.52,26.00,26.71,5.36,1.95


In [ ]:
#apparently, vitals are taken on one of two machines - carevue or metavision. combine the results...

#when there's values in both metavisions, average them and replace the _2 with the result
df.loc[df['mv_diastolic_blood_pressure_1'].notna() & df['mv_diastolic_blood_pressure_2'].notna(),
        'mv_diastolic_blood_pressure_2'] = round((df['mv_diastolic_blood_pressure_1'] + df['mv_diastolic_blood_pressure_2']) / 2,2)
df.loc[df['mv_mean_arterial_pressure_1'].notna() & df['mv_mean_arterial_pressure_2'].notna(),
        'mv_mean_arterial_pressure_2'] = round((df['mv_mean_arterial_pressure_1'] + df['mv_mean_arterial_pressure_2']) / 2,2)
df.loc[df['mv_systolic_blood_pressure_1'].notna() & df['mv_systolic_blood_pressure_2'].notna(),
        'mv_systolic_blood_pressure_2'] = round((df['mv_systolic_blood_pressure_1'] + df['mv_systolic_blood_pressure_2']) / 2,2)

In [ ]:
#move mv values over to _2 if there's one in _1 but not in _2
df.loc[df['mv_diastolic_blood_pressure_1'].notna() & df['mv_diastolic_blood_pressure_2'].isna(),
       'mv_diastolic_blood_pressure_2'] = df['mv_diastolic_blood_pressure_1']
df.loc[df['mv_mean_arterial_pressure_1'].notna() & df['mv_mean_arterial_pressure_2'].isna(),
       'mv_mean_arterial_pressure_2'] = df['mv_mean_arterial_pressure_1']
df.loc[df['mv_systolic_blood_pressure_1'].notna() & df['mv_systolic_blood_pressure_2'].isna(),
       'mv_systolic_blood_pressure_2'] = df['mv_systolic_blood_pressure_1']

In [ ]:
#if there's values in mv and cv, average them and dump the result into cv
#do this for mv_heart_rate mv_respiratory_rate mv_body_temp mv_diastolic_blood_pressure_2 mv_mean_arterial_pressure_2 mv_systolic_blood_pressure_2
#do this for cv_heart_rate cv_respiratory_rate cv_body_temp cv_diastolic_blood_pressure cv_mean_arterial_pressure cv_systolic_blood_pressure
df.loc[df['mv_heart_rate'].notna() & df['cv_heart_rate'].notna(),
        'cv_heart_rate'] = round((df['mv_heart_rate'] + df['cv_heart_rate']) / 2,2)
df.loc[df['mv_respiratory_rate'].notna() & df['cv_respiratory_rate'].notna(),
        'cv_respiratory_rate'] = round((df['mv_respiratory_rate'] + df['cv_respiratory_rate']) / 2,2)
df.loc[df['mv_body_temp'].notna() & df['cv_body_temp'].notna(),
        'cv_body_temp'] = round((df['mv_body_temp'] + df['cv_body_temp']) / 2,2)
df.loc[df['mv_diastolic_blood_pressure_2'].notna() & df['cv_diastolic_blood_pressure'].notna(),
        'cv_diastolic_blood_pressure'] = round((df['mv_diastolic_blood_pressure_2'] + df['cv_diastolic_blood_pressure']) / 2,2)
df.loc[df['mv_mean_arterial_pressure_2'].notna() & df['cv_mean_arterial_pressure'].notna(),
        'cv_mean_arterial_pressure'] = round((df['mv_mean_arterial_pressure_2'] + df['cv_mean_arterial_pressure']) / 2,2)
df.loc[df['mv_systolic_blood_pressure_2'].notna() & df['cv_systolic_blood_pressure'].notna(),
        'cv_systolic_blood_pressure'] = round((df['mv_systolic_blood_pressure_2'] + df['cv_systolic_blood_pressure']) / 2,2)

In [ ]:
#move values over to cv if there's one in mv but not in cv
#do this for mv_heart_rate mv_respiratory_rate mv_body_temp mv_diastolic_blood_pressure_2 mv_mean_arterial_pressure_2 mv_systolic_blood_pressure_2
#do this for cv_heart_rate cv_respiratory_rate cv_body_temp cv_diastolic_blood_pressure cv_mean_arterial_pressure cv_systolic_blood_pressure
df.loc[df['mv_heart_rate'].notna() & df['cv_heart_rate'].isna(), 'cv_heart_rate'] = df['mv_heart_rate']
df.loc[df['mv_respiratory_rate'].notna() & df['cv_respiratory_rate'].isna(), 'cv_respiratory_rate'] = df['mv_respiratory_rate']
df.loc[df['mv_body_temp'].notna() & df['cv_body_temp'].isna(), 'cv_body_temp'] = df['mv_body_temp']
df.loc[df['mv_diastolic_blood_pressure_2'].notna() & df['cv_diastolic_blood_pressure'].isna(), 'cv_diastolic_blood_pressure'] = df['mv_diastolic_blood_pressure_2']
df.loc[df['mv_mean_arterial_pressure_2'].notna() & df['cv_mean_arterial_pressure'].isna(), 'cv_mean_arterial_pressure'] = df['mv_mean_arterial_pressure_2']
df.loc[df['mv_systolic_blood_pressure_2'].notna() & df['cv_systolic_blood_pressure'].isna(), 'cv_systolic_blood_pressure'] = df['mv_systolic_blood_pressure_2']

In [ ]:
#now can drop all the mv's
droplist = ['mv_diastolic_blood_pressure_1','mv_diastolic_blood_pressure_2','mv_mean_arterial_pressure_1','mv_mean_arterial_pressure_2','mv_systolic_blood_pressure_1','mv_systolic_blood_pressure_2','mv_body_temp','mv_heart_rate','mv_respiratory_rate']
df = df.drop(columns=droplist)
df.head()

,gender_binaried,inhospital_death,diabetes,chronic_kidney_disease,chronic_ischemic_heart_disease,anion_gap,albumin,bicarbonate,bilirubin_total,creatinine,chloride,glucose_bloodgas,hematocrit_blood,hemoglobin_bloodgas,lactate,platelet_count,potassium_blood,ptt,inr_pt,sodium,bun,wbc_blood,calcium_total,free_calcium,cv_heart_rate,cv_respiratory_rate,cv_body_temp,cv_systolic_blood_pressure,cv_diastolic_blood_pressure,cv_mean_arterial_pressure,spo2
0,1,1,1,0,1,22.00,3.00,16.00,0.40,3.20,105.00,128.00,32.00,9.20,3.90,310.00,5.10,27.10,1.40,138.00,83.00,17.50,9.20,1.17,84.00,21.00,98.80,107.00,49.00,90.00,100.00
1,0,0,0,0,0,13.00,3.70,27.00,0.30,1.00,106.00,100.00,39.20,12.50,1.60,301.00,4.10,25.80,1.40,142.00,26.00,9.20,9.40,1.02,92.00,12.00,98.10,118.00,69.00,85.00,100.00
2,1,0,0,0,0,19.00,3.50,13.00,0.70,3.10,105.00,120.00,28.80,NaN,0.80,198.00,3.30,29.80,1.20,134.00,46.00,7.20,9.70,1.29,95.00,34.00,99.90,175.00,81.00,108.00,100.00
3,1,0,0,0,0,13.00,2.00,26.00,0.50,0.90,103.00,160.00,30.60,10.20,1.10,40.00,3.30,29.30,1.20,139.00,19.00,21.00,8.70,1.05,122.00,34.00,100.80,120.00,56.00,99.00,93.00
4,0,0,1,0,1,20.00,3.70,25.00,0.30,1.10,94.00,373.00,38.90,NaN,2.60,561.00,4.50,34.00,1.20,134.00,27.00,10.80,9.60,NaN,100.00,21.00,100.50,NaN,NaN,NaN,98.00


In [ ]:
#rename the cv columns cuz they're not only carevue anymore
#cv_heart_rate	cv_respiratory_rate	cv_body_temp cv_systolic_blood_pressure	cv_diastolic_blood_pressure	cv_mean_arterial_pressure
df = df.rename(columns={
    'cv_heart_rate': 'heart_rate',
    'cv_respiratory_rate': 'respiratory_rate',
    'cv_body_temp': 'body_temperature',
    'cv_systolic_blood_pressure': 'systolic_blood_pressure',
    'cv_diastolic_blood_pressure': 'diastolic_blood_pressure',
    'cv_mean_arterial_pressure': 'mean_arterial_pressure'
})
df.head()

,gender_binaried,inhospital_death,diabetes,chronic_kidney_disease,chronic_ischemic_heart_disease,anion_gap,albumin,bicarbonate,bilirubin_total,creatinine,chloride,glucose_bloodgas,hematocrit_blood,hemoglobin_bloodgas,lactate,platelet_count,potassium_blood,ptt,inr_pt,sodium,bun,wbc_blood,calcium_total,free_calcium,heart_rate,respiratory_rate,body_temperature,systolic_blood_pressure,diastolic_blood_pressure,mean_arterial_pressure,spo2
0,1,1,1,0,1,22.00,3.00,16.00,0.40,3.20,105.00,128.00,32.00,9.20,3.90,310.00,5.10,27.10,1.40,138.00,83.00,17.50,9.20,1.17,84.00,21.00,98.80,107.00,49.00,90.00,100.00
1,0,0,0,0,0,13.00,3.70,27.00,0.30,1.00,106.00,100.00,39.20,12.50,1.60,301.00,4.10,25.80,1.40,142.00,26.00,9.20,9.40,1.02,92.00,12.00,98.10,118.00,69.00,85.00,100.00
2,1,0,0,0,0,19.00,3.50,13.00,0.70,3.10,105.00,120.00,28.80,NaN,0.80,198.00,3.30,29.80,1.20,134.00,46.00,7.20,9.70,1.29,95.00,34.00,99.90,175.00,81.00,108.00,100.00
3,1,0,0,0,0,13.00,2.00,26.00,0.50,0.90,103.00,160.00,30.60,10.20,1.10,40.00,3.30,29.30,1.20,139.00,19.00,21.00,8.70,1.05,122.00,34.00,100.80,120.00,56.00,99.00,93.00
4,0,0,1,0,1,20.00,3.70,25.00,0.30,1.10,94.00,373.00,38.90,NaN,2.60,561.00,4.50,34.00,1.20,134.00,27.00,10.80,9.60,NaN,100.00,21.00,100.50,NaN,NaN,NaN,98.00


In [ ]:
cstat(df, '# Count')

,Feature,Dtype,# Count,# NaN,# Unique,# Dupes,Min,Max,% NaN,% Zero,Mean,Median,Std Dev,Kurtosis (3),Skew (0)
13,hemoglobin_bloodgas,float64,593,701,102,81,3.40,20.00,54.17,0.00,10.86,10.70,2.18,0.65,0.38
11,glucose_bloodgas,float64,697,597,220,133,14.00,1105.00,46.14,0.00,152.72,132.00,94.75,30.74,4.56
29,mean_arterial_pressure,float64,707,587,146,85,0.00,361.00,45.36,0.54,82.31,80.00,36.90,18.26,2.98
27,systolic_blood_pressure,float64,717,577,149,101,0.00,223.00,44.59,3.01,117.11,120.00,39.11,2.42,-1.11
28,diastolic_blood_pressure,float64,730,564,94,65,0.00,128.00,43.59,3.09,57.66,59.00,19.62,2.44,-0.92
30,spo2,float64,785,509,27,18,0.00,100.00,39.34,0.15,96.67,98.00,7.53,101.55,-9.01
23,free_calcium,float64,821,473,71,54,0.09,1.84,36.55,0.00,1.10,1.11,0.12,9.81,-0.51
14,lactate,float64,1086,208,95,65,0.30,16.50,16.07,0.00,2.35,1.80,1.90,12.58,3.00
6,albumin,float64,1097,197,39,35,1.00,5.00,15.22,0.00,2.96,3.00,0.68,-0.34,0.07
8,bilirubin_total,float64,1098,196,92,43,0.00,82.80,15.15,0.15,1.34,0.50,3.88,202.51,11.94


In [ ]:
df.to_csv('F:/Python/project 650/_CDI.Training.cleaned.csv', index=False)